In [1]:
import numpy as np
import os   # 운영체제와 상호작용하기 위한 모듈
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
base_dir = os.getenv('BASE_DIR')

In [2]:
# 기존에 학습시킨 데이터
data = np.concatenate([
    np.load(os.path.join(base_dir, f'seq_data/20words/seq_npy_1727115433_20_10.npy')),
    np.load(os.path.join(base_dir, f'seq_data/20words/seq_npy_flip_1727115434_20_10.npy')),
    np.load(os.path.join(base_dir, f'seq_data/20words/seq_npy_shift_1727115434_20_10.npy')),
    np.load(os.path.join(base_dir, f'seq_data/20words/seq_npy_flip_shift_1727115435_20_10.npy')),
], axis=0)

# 추가 학습시킬 데이터
new_data = np.concatenate([
    np.load(os.path.join(base_dir, f'ft_seq/seq_ft_1727487535_20_10.npy')),
    np.load(os.path.join(base_dir, f'ft_seq/seq_ft_flip_1727487535_20_10.npy')),
    np.load(os.path.join(base_dir, f'ft_seq/seq_ft_shift_1727487535_20_10.npy')),
    np.load(os.path.join(base_dir, f'ft_seq/seq_ft_flip_shift_1727487535_20_10.npy')),
], axis=0)

print(data.shape)
print(new_data.shape)
# (프레임 수, 시퀀스 길이, 한 프레임당 데이터 개수)

(13656, 20, 235)
(1932, 20, 235)


In [3]:
from setting import actions
from keras.utils import to_categorical

# 기존 데이터
# 데이터 분리 및 전처리
x_data = data[:, :, :-1]    # 시퀀스의 마지막 요소 제외한 모든 값 가져와 할당
labels = data[:, 0, -1]     # 마지막 요소는 레이블 값
y_data = to_categorical(labels, num_classes=len(actions))   # 원-핫 인코딩으로 변환

# 추가 학습 시킬 데이터
# 데이터 분리 및 전처리
new_x = new_data[:, :, :-1]    # 시퀀스의 마지막 요소 제외한 모든 값 가져와 할당
new_labels = new_data[:, 0, -1]     # 마지막 요소는 레이블 값
new_y = to_categorical(new_labels, num_classes=len(actions))    # 원-핫 인코딩으로 변환

print(new_x.shape)
print(new_labels.shape)
print(new_y.shape)         # y_data 형태 -> [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...]
print(np.unique(new_labels))    # 레이블 값 출력


(1932, 20, 234)
(1932,)
(1932, 20)
[ 3. 10. 14.]


In [5]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.models import load_model
from keras.layers import Dense

# 기존 모델 로드
model = load_model('models/model_20words_90.keras')

# 모델 추가 학습
history = model.fit(
    new_x, new_y,
    validation_data=(x_data, y_data),
    epochs=100,
    batch_size=32,
    callbacks=[
        ModelCheckpoint('models/model.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, verbose=1, mode='auto'),
        # EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True),
    ]
)

Epoch 1/100


61/61 [==============================] - ETA: 0s - loss: 0.0411 - accuracy: 0.9896
Epoch 1: val_accuracy improved from -inf to 0.95496, saving model to models\model.keras
61/61 [==============================] - 11s 149ms/step - loss: 0.0411 - accuracy: 0.9896 - val_loss: 0.2467 - val_accuracy: 0.9550 - lr: 0.0010
Epoch 2/100
60/61 [============================>.] - ETA: 0s - loss: 0.0521 - accuracy: 0.9859
Epoch 2: val_accuracy improved from 0.95496 to 0.96331, saving model to models\model.keras
61/61 [==============================] - 8s 129ms/step - loss: 0.0523 - accuracy: 0.9855 - val_loss: 0.1517 - val_accuracy: 0.9633 - lr: 0.0010
Epoch 3/100
61/61 [==============================] - ETA: 0s - loss: 0.0203 - accuracy: 0.9928
Epoch 3: val_accuracy did not improve from 0.96331
61/61 [==============================] - 7s 121ms/step - loss: 0.0203 - accuracy: 0.9928 - val_loss: 0.2377 - val_accuracy: 0.9513 - lr: 0.0010
Epoch 4/100
61/61 [==============================]

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# 학습 데이터/검증 데이터 정확도/손실 그래프
fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['accuracy'], 'b', label='train accuracy')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val accuracy')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper right')

plt.show()